In [ ]:
pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

[NEW] Llama-3.1 8b, 70b & 405b are trained on a crazy 15 trillion tokens with 128K long context lengths!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
* [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

# **낚시 판별만**

In [ ]:
classification_prompt = """Classify the following news article as fishing or non-fishing based on its title and content.

### Title:
{}

### Content:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Ensure EOS_TOKEN is added

def formatting_prompts_func(examples):
    titles = examples["newsTitle"]
    contents = examples["newsContent"]
    classes = examples["clickbaitClass"]
    texts = []
    for title, content, cls in zip(titles, contents, classes):
        # Set the response based on clickbaitClass
        response = "fishing article" if cls == 0 else "non-fishing article"
        text = classification_prompt.format(title, content, response) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}


pass

from datasets import Dataset

# Load the custom JSON dataset
dataset = Dataset.from_json("combined_data.json")
dataset = dataset.map(formatting_prompts_func, batched=True)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 6250,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
6.004 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,102 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 6,250
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 20211305 (20211305-). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.112400
2,2.149700
3,2.253700
4,2.092400
5,2.060600
6,2.021200
7,2.001800
8,1.804000
9,2.009600
10,2.034600


Step,Training Loss
1,2.112400
2,2.149700
3,2.253700
4,2.092400
5,2.060600
6,2.021200
7,2.001800
8,1.804000
9,2.009600
10,2.034600


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

19322.2028 seconds used for training.
322.04 minutes used for training.
Peak reserved memory = 7.701 GB.
Peak reserved memory for training = 1.697 GB.
Peak reserved memory % of max memory = 19.465 %.
Peak reserved memory for training % of max memory = 4.289 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
# alpaca_prompt = Copied from above
#정답 : fishing article
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    classification_prompt.format(
        "한겨레, \\\"전작권 엠바고 애초 성립하지 않았다\\\"", # Title
        "한미 정상이 전시작전통제권(전작권) 전환 시점 연기를 논의 중이라는 한겨레 보도가 엠바고를 깬 것이라는 이동관 청와대 홍보수석의 주장에 대해 한겨레가 \\\"사실을 왜곡한 것\\\"이라며 \\\"이 사안은 엠바고(특정시점까지 보도를 유예하는 것)가 성립하지 않았다\\\"고 반박했다.\n한겨레는 애초 엠바고에 동의하지 않았다는 것이다.\n이날 전작권 환수 재논의는 경향신문도 23일 1면 를 통해 보도한 바 있다.\n한겨레는 24일자 1면 기사를 통해 \\\"한미 양국 정상이 오는 26일 캐나다 토론토에서 열리는 정상회담에서 전작권 전환을 연기하는 문제를 공식 논의하기로 했다는 소식이 23일 알려지자 정치권과 시민사회가 일제히 반발하고 나섰다\\\"고 보도한 바 있다.\n이에 대해 이동관 홍보수석은 24일 국회 운영위원회에 출석해 \\\"한미 정상회담에서 전작권 연기 문제가 논의될 것이 분명하지 않느냐\\\"는 이정희 민주노동당 의원의 질의에 \\\"그렇게 될 가능성이 있기 때문에, 그러나 민감한 사안이므로 엠바고를 지켜달라고 부탁드렸는데 한겨레가 깨고 혼자 기사를 쓴 것\\\"이라고 말했다.\n한겨레는 이에 대해 25일자 2면 을 통해 \\\"청와대는 지난 22일 오후 이 사안에 대한 엠바고를 청와대 출입기자단에 요청해 왔으나 한겨레는 검토 끝에 '엠바고를 수용할 수 없다'는 방침을 전달했다\\\"며 \\\"아울러 23일치 신문에 전작권 전환 연기 관련 보도를 하겠다는 뜻도 통보했다\\\"고 밝혔다.\n당시 이 수석은 엠바고 수용 여부에 대한 기자들 합의가 이뤄지기도 전에 엠바고 성립을 기정사실로 전제하고 \\\"엠바고를 지켜달라\\\"고 말했다는 것이다.\n엠바고를 수용하지 않은 것에 대해 한겨레는 \\\"전작권 문제는 국민의 의견이 갈려 있는 만큼 공론화되어야 하며 그러려면 관련 내용에 대한 시의적절하고 충실한 보도가 필요하다고 판단했기 때문\\\"이라며 \\\"더구나 전작권 논의를 부인해온 청와대가 정상회담에 임박해 그동안의 태도를 바꾸며 엠바고를 요청한 것은 정부의 책임 있는 자세가 아니\\\"라고 지적했다.", # Content
        "" # Leave response blank for generation
    )
], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.batch_decode(outputs))

['<|begin_of_text|>Classify the following news article as fishing or non-fishing based on its title and content.\n\n### Title:\n한겨레, \\"전작권 엠바고 애초 성립하지 않았다\\"\n\n### Content:\n한미 정상이 전시작전통제권(전작권) 전환 시점 연기를 논의 중이라는 한겨레 보도가 엠바고를 깬 것이라는 이동관 청와대 홍보수석의 주장에 대해 한겨레가 \\"사실을 왜곡한 것\\"이라며 \\"이 사안은 엠바고(특정시점까지 보도를 유예하는 것)가 성립하지 않았다\\"고 반박했다.\n한겨레는 애초 엠바고에 동의하지 않았다는 것이다.\n이날 전작권 환수 재논의는 경향신문도 23일 1면 를 통해 보도한 바 있다.\n한겨레는 24일자 1면 기사를 통해 \\"한미 양국 정상이 오는 26일 캐나다 토론토에서 열리는 정상회담에서 전작권 전환을 연기하는 문제를 공식 논의하기로 했다는 소식이 23일 알려지자 정치권과 시민사회가 일제히 반발하고 나섰다\\"고 보도한 바 있다.\n이에 대해 이동관 홍보수석은 24일 국회 운영위원회에 출석해 \\"한미 정상회담에서 전작권 연기 문제가 논의될 것이 분명하지 않느냐\\"는 이정희 민주노동당 의원의 질의에 \\"그렇게 될 가능성이 있기 때문에, 그러나 민감한 사안이므로 엠바고를 지켜달라고 부탁드렸는데 한겨레가 깨고 혼자 기사를 쓴 것\\"이라고 말했다.\n한겨레는 이에 대해 25일자 2면 을 통해 \\"청와대는 지난 22일 오후 이 사안에 대한 엠바고를 청와대 출입기자단에 요청해 왔으나 한겨레는 검토 끝에 \'엠바고를 수용할 수 없다\'는 방침을 전달했다\\"며 \\"아울러 23일치 신문에 전작권 전환 연기 관련 보도를 하겠다는 뜻도 통보했다\\"고 밝혔다.\n당시 이 수석은 엠바고 수용 여부에 대한 기자들 합의가 이뤄지기도 전에 엠바고 성립을 기정사실로 전제하고 \\"엠바고를 지켜달라\\"고 말했다는 것이다.\n엠바고를 수용하지 

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
# 정답 : fishing article
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    classification_prompt.format(
        "YTN 노사 임단협 타결", # Title
        "YTN 노사가 7개월 여 공전 끝에 2010년 임금단체협상을 타결했다.\n전국언론노동조합 YTN지부(지부장 유투권)와 YTN(사장 배석규) 경영진은 31일 오후 교섭안에 합의했으며, 노동조합은 오는 1일 대의원대회에서 이를 추인 받을 계획이다.\n앞서 YTN 노사는 지난해 11월부터 교섭을 벌였으나 난항을 거듭하다 지난달 13일 서울지방노동위원회에 조정 신청을 접수했지만 이 역시 최종결렬 된 바 있다.\n이후 노사는 최근까지 막판교섭을 통해 △보도국장 선출방식 △공정방송위원회 현실화 문제 △쟁의기간 인사조처 등 단협 사안을 놓고 협상을 벌여왔다.\n31일 노사합의에 따라, 가장 큰 쟁점이었던 YTN 공정방송위원회는 지난해 6월 체결한 공정방송협약 내용을 유지하되 '협약 개정'을 요구해온 사쪽 주장을 감안해 다른 통로로 노사가 공방위 활동에 준하는 노력을 기울이기로 했다.\n노사는 구체적 협약내용을 곧 발표할 예정이다.", # Content
        "" # Leave response blank for generation
    )
], return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Classify the following news article as fishing or non-fishing based on its title and content.

### Title:
YTN 노사 임단협 타결

### Content:
YTN 노사가 7개월 여 공전 끝에 2010년 임금단체협상을 타결했다.
전국언론노동조합 YTN지부(지부장 유투권)와 YTN(사장 배석규) 경영진은 31일 오후 교섭안에 합의했으며, 노동조합은 오는 1일 대의원대회에서 이를 추인 받을 계획이다.
앞서 YTN 노사는 지난해 11월부터 교섭을 벌였으나 난항을 거듭하다 지난달 13일 서울지방노동위원회에 조정 신청을 접수했지만 이 역시 최종결렬 된 바 있다.
이후 노사는 최근까지 막판교섭을 통해 △보도국장 선출방식 △공정방송위원회 현실화 문제 △쟁의기간 인사조처 등 단협 사안을 놓고 협상을 벌여왔다.
31일 노사합의에 따라, 가장 큰 쟁점이었던 YTN 공정방송위원회는 지난해 6월 체결한 공정방송협약 내용을 유지하되 '협약 개정'을 요구해온 사쪽 주장을 감안해 다른 통로로 노사가 공방위 활동에 준하는 노력을 기울이기로 했다.
노사는 구체적 협약내용을 곧 발표할 예정이다.

### Response:
fishing article<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [1]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!
# 정답 : non-fishing article
inputs = tokenizer(
[
    classification_prompt.format(
        "보성 율포에 세계 최장 스킨스쿠버풀 조성...해양복합센터 용역 착수", # Title
        "전남 보성 율포해수욕장 인근에 사계절 즐길 수 있는 세계 최장 45m 깊이의 스킨스쿠버풀이 들어선다.\n보성군은 최근 김철우 보성군수, 관계 전문가 등이 참석한 가운데 율포해양복합센터 설계용역 착수 보고회를 개최했다고 4일 밝혔다.\n주요 시설로는 세계 최장 깊이 45m 스킨스쿠버풀을 비롯해 생존체험장, 실내 서핑장, 수중 스튜디오 등이 들어설 예정이다.\n또 인피니티 풀과 국내 최장 길이의 스카이워크 등이며 사계절 해양문화를 체험할 수 있는 복합시설도 구축할 계획이다.\n율포해수욕장 일원은 사계절 온화한 기후가 특징이며, 자연경관이 우수한 천혜의 입지조건을 자랑한다.\n율포 종합 관광지가 있어 기본 관광 인프라가 구축돼 있고, 보성차밭 등 주요 관광지와 인접해 있어 관광자원 활용도도 높은 지역이다.\n보성군은 설계 용역을 거쳐 연내에 사업을 착공하고, 2024년 완공한다는 목표다.\n또 추가로 해양 SOC사업을 유치하고, 동율항·군학항 등에서 추진되고 있는 어촌 뉴딜300 사업 등과 연계해 사업의 시너지를 높일 방침이다.\n해양레저 거점사업은 해양수산부 공모사업으로 2020년 전국에서 2곳이 선정됐으며, 총 사업비 430억원이 투입되는 대형 사업이다.\n김철우 보성군수는 “해양레저 관광거점 사업으로 율포 해변을 남해안 대표 해변으로 육성하겠다”면서 “추가 사업 등을 유치해 주변 지역도 규모를 갖춘 관광지로 조성하겠다”고 말했다.", # Content
        "" # Leave response blank for generation
    )
], return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

NameError: name 'tokenizer' is not defined

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
import wandb
import os

# Initialize WandB
run = wandb.init(project="fishing-article-classification_6250")

# Create the artifact
artifact = wandb.Artifact('fishing_article_model_6250', type='model')

# Set the correct path to the lora_model folder
model_dir = "lora_model"  # Path to your model folder (change it to 'lora_model')

# Check if the directory exists
if os.path.isdir(model_dir):
    artifact.add_dir(model_dir)  # Add the model directory to the artifact
    wandb.log_artifact(artifact)
else:
    print(f"The directory {model_dir} does not exist or is not a directory.")


wandb: Adding directory to artifact (./lora_model)... Done. 0.6s


Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>

# **성능 테스트**

In [ ]:
pip install transformers datasets peft torch

# **Perplexity**

In [ ]:
import math
import random

def calculate_perplexity_with_shuffle(model, tokenizer, dataset, num_samples=100):
    total_log_likelihood = 0  # To accumulate the log likelihood for perplexity calculation
    total = 0

    # Shuffle the dataset and select num_samples
    shuffled_dataset = dataset.shuffle(seed=random.randint(0, 5000))  # Shuffle with a random seed
    limited_dataset = shuffled_dataset.select(range(num_samples))

    for example in limited_dataset:
        # Format the prompt with the news title and content
        prompt = classification_prompt.format(example["newsTitle"], example["newsContent"], "")

        # Tokenize the input and prepare for the model
        inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=2048).to("cuda")

        # Calculate log likelihood for perplexity
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])  # Use labels for loss calculation
            log_likelihood = outputs.loss.item()  # Log likelihood
            total_log_likelihood += log_likelihood

        total += 1

    # Calculate perplexity: exp(log_likelihood / num_samples)
    perplexity = math.exp(total_log_likelihood / total) if total > 0 else float('inf')

    return perplexity


In [ ]:
# Load the dataset
from datasets import Dataset

# Load your dataset from JSON file
dataset = Dataset.from_json("combined_validdata.json")

# Calculate perplexity with shuffled dataset
perplexity = calculate_perplexity_with_shuffle(model, tokenizer, dataset, num_samples=5000)

# Print the result
print(f"Perplexity: {perplexity:.2f}")


Perplexity: 5.50


# **precision**

In [ ]:
def calculate_precision(model, tokenizer, dataset, num_samples=100):
    import random

    # 섞은 데이터셋에서 num_samples 만큼 선택
    shuffled_dataset = dataset.shuffle(seed=random.randint(0, 5000))
    limited_dataset = shuffled_dataset.select(range(num_samples))

    tp = fp = 0

    # Helper functions
    def normalize_response(response):
        return response.lower().strip().replace(".", "").replace(",", "")

    def extract_prediction(prediction):
        if "fishing" in prediction:
            return "fishing"
        elif "non-fishing" in prediction:
            return "non-fishing"
        else:
            return "unknown"

    # Evaluate predictions
    for example in limited_dataset:
        prompt = classification_prompt.format(example["newsTitle"], example["newsContent"], "")
        inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=2048).to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=10, use_cache=True)
        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip().lower()

        ground_truth = "fishing" if example["clickbaitClass"] == 0 else "non-fishing"
        normalized_ground_truth = normalize_response(ground_truth)
        normalized_prediction = normalize_response(extract_prediction(prediction))

        if normalized_prediction == "unknown":
            print(f"Unrecognized prediction: {prediction}")
            continue

        if normalized_prediction == "fishing":
            if normalized_ground_truth == "fishing":
                tp += 1
            else:
                fp += 1

    # Calculate precision: TP / (TP + FP)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    print(f"TP: {tp}, FP: {fp}")
    return precision


In [ ]:
# Load the test dataset
from datasets import Dataset

# Load the test dataset from JSON file
dataset = Dataset.from_json("combined_validdata.json")

# Calculate and print precision
precision = calculate_precision(model, tokenizer, dataset, num_samples=5000)
print(f"Precision: {precision * 100:.2f}%")

TP: 2166, FP: 2834
Precision: 43.32%


# **Accuracy**

In [ ]:
def calculate_accuracy(model, tokenizer, dataset, num_samples=100):
    import random

    # 섞은 데이터셋에서 num_samples 만큼 선택
    shuffled_dataset = dataset.shuffle(seed=random.randint(0, 5000))
    limited_dataset = shuffled_dataset.select(range(num_samples))

    correct = total = 0

    # Helper functions
    def normalize_response(response):
        return response.lower().strip().replace(".", "").replace(",", "")

    def extract_prediction(prediction):
        if "fishing" in prediction:
            return "fishing"
        elif "non-fishing" in prediction:
            return "non-fishing"
        else:
            return "unknown"

    # Evaluate predictions
    for example in limited_dataset:
        prompt = classification_prompt.format(example["newsTitle"], example["newsContent"], "")
        inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=2048).to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=10, use_cache=True)
        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip().lower()

        ground_truth = "fishing" if example["clickbaitClass"] == 0 else "non-fishing"
        normalized_ground_truth = normalize_response(ground_truth)
        normalized_prediction = normalize_response(extract_prediction(prediction))

        if normalized_prediction == "unknown":
            print(f"Unrecognized prediction: {prediction}")
            continue

        if normalized_ground_truth == normalized_prediction:
            correct += 1

        total += 1

    # Calculate accuracy: Correct / Total
    accuracy = correct / total if total > 0 else 0
    print(f"Correct: {correct}, Total: {total}")
    return accuracy


In [ ]:
# Load the test dataset
from datasets import Dataset

# Load the test dataset from JSON file
dataset = Dataset.from_json("combined_data.json")

In [ ]:
# Calculate and print accuracy
accuracy = calculate_accuracy(model, tokenizer, dataset, num_samples=5000)
print(f"Accuracy: {accuracy * 100:.2f}%")

Correct: 2284, Total: 5000
Accuracy: 45.68%
